# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,san patricio,28.0170,-97.5169,21.20,44,0,1.66,US,1707249185
1,1,st. john's,47.5649,-52.7093,0.93,97,100,15.95,CA,1707249118
2,2,keflavik,64.0049,-22.5624,-2.05,74,40,7.72,IS,1707249186
3,3,port-aux-francais,-49.3500,70.2167,6.02,89,100,11.78,TF,1707249186
4,4,ribeira grande,38.5167,-28.7000,17.10,94,75,13.89,PT,1707249186


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    scale = 1.20, 
    title = "Humidity Heat Map", 
    size = "Humidity", 
    color = "City"
)


# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 29) 
                                             &(city_data_df["Max Temp"] > 20) 
                                             &(city_data_df["Wind Speed"] < 10)
                                             &(city_data_df["Cloudiness"] == 0), 
:].dropna().reset_index()

# Display sample data
ideal_weather_df.head()

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,0,san patricio,28.0170,-97.5169,21.20,44,0,1.66,US,1707249185
1,62,62,nova sintra,14.8667,-24.7167,21.14,72,0,3.90,CV,1707249199
2,98,98,tazacorte,28.6290,-17.9293,20.08,49,0,2.68,ES,1707249207
3,108,108,pathein,16.7833,94.7333,22.58,67,0,4.51,MM,1707249074
4,135,135,iskushuban,10.2837,50.2300,22.20,82,0,4.51,SO,1707249214


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,san patricio,US,28.0170,-97.5169,44,
1,nova sintra,CV,14.8667,-24.7167,72,
2,tazacorte,ES,28.6290,-17.9293,49,
3,pathein,MM,16.7833,94.7333,67,
4,iskushuban,SO,10.2837,50.2300,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
limit = 20
radius = 10000
params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san patricio - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
tazacorte - nearest hotel: App Leyma
pathein - nearest hotel: Taan Taan Ta
iskushuban - nearest hotel: No hotel found
el campo - nearest hotel: Days Inn by Wyndham El Campo TX
teocelo - nearest hotel: No hotel found
dabakala - nearest hotel: Hôtel Résidence les villages
saint-pierre - nearest hotel: Tropic Hotel
coquimbo - nearest hotel: Hotel Iberia
san julian - nearest hotel: No hotel found
castillos - nearest hotel: A mi gente
silsbee - nearest hotel: No hotel found
bubaque - nearest hotel: Chez Julio
yamethin - nearest hotel: No hotel found
freetown - nearest hotel: Formal Leona hotel
holualoa - nearest hotel: Kona Hotel
ojinaga - nearest hotel: Hotel
san blas - nearest hotel: Suites San Blas
pyapon - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
robertsport - nearest hotel: Nana's Lodge
richards bay - nearest hotel: SUN1 Richards Bay
myan

,City,Country,Lat,Lng,Humidity,Hotel Name
0,san patricio,US,28.0170,-97.5169,44,No hotel found
1,nova sintra,CV,14.8667,-24.7167,72,Residência Ka Dencho
2,tazacorte,ES,28.6290,-17.9293,49,App Leyma
3,pathein,MM,16.7833,94.7333,67,Taan Taan Ta
4,iskushuban,SO,10.2837,50.2300,82,No hotel found
5,el campo,US,29.1966,-96.2697,32,Days Inn by Wyndham El Campo TX
6,teocelo,MX,19.3833,-96.9667,48,No hotel found
7,dabakala,CI,8.3667,-4.4333,16,Hôtel Résidence les villages
8,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
9,coquimbo,CL,-29.9533,-71.3436,68,Hotel Iberia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
city_map_enhanced = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    tiles = "OSM", 
    frame_width = 700, 
    frame_height = 500, 
    scale = 1.20, 
    title = "Humidity Heat Map (with hover)", 
    hover_cols = ["Country","Hotel Name"], 
    size = "Humidity", 
    color = "City"
)

# Display the map
city_map_enhanced

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)